<a href="https://colab.research.google.com/github/Abeeha-Fatima/quater2_projects/blob/main/Q2_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU langchain-pinecone langchain-google-genai # install and setup pinecone and langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.9 MB/s eta 0:00:00


In [3]:
from pinecone import Pinecone,ServerlessSpec
import os
from google.colab import userdata
os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')
pc = Pinecone()

In [4]:
index_name = "rag-project" # initialize index

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists.")

index = pc.Index(index_name)

Index 'rag-project' created successfully.


In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings # setup embeddings
from google.auth.credentials import AnonymousCredentials
import os

credentials = userdata.get('GEMINI_API_KEY')

os.environ['GEMINI_API_KEY'] = credentials

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=credentials,
)

vector = embeddings.embed_query("we are building a RAG Text")

print(vector[:5])

[0.04060795158147812, -0.04385491460561752, -0.05683692544698715, -0.021194666624069214, 0.010449625551700592]


In [7]:
from langchain_pinecone import PineconeVectorStore # setup vector store
vector_store = PineconeVectorStore(
    index=index,
    embedding=embeddings
    )

In [29]:
from langchain_core.documents import Document # setup and upload documents

# Define each document
document_1 = Document(
    page_content="Dogs are great companions, known for their loyalty and friendliness.",
    metadata={"source": "mammal-pets-doc"},
)
document_2 = Document(
    page_content="Cats are independent pets that often enjoy their own space.",
    metadata={"source": "mammal-pets-doc"},
)
document_3 = Document(
    page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
    metadata={"source": "fish-pets-doc"},
)
document_4 = Document(
    page_content="Parrots are intelligent birds capable of mimicking human speech.",
    metadata={"source": "bird-pets-doc"},
)
document_5 = Document(
    page_content="Rabbits are social animals that need plenty of space to hop around.",
    metadata={"source": "mammal-pets-doc"},
)

# Create the list
documents = [document_1, document_2, document_3, document_4, document_5]


In [46]:
user_id = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents,id=user_id)

['9d2af9ec-fbe6-4e25-8bc0-7f3ac482e74c',
 'a3a26601-529d-43dd-80ce-609470a60b0c',
 '005b2144-ae0a-496e-82b5-4132bd50d804',
 '9aea77f9-c2bd-429b-852b-13f27299b225',
 '988cae84-ff9a-46ef-a2fb-0ffc6a2e3091']

In [34]:
# setup retriver
results = vector_store.similarity_search(
   "Goldfish are popular pets for beginners",
    k=3,
    filter={"source": "fish-pets-doc"},
)

# Iterating over the results and printing the content of each matching document along with its metadata
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Goldfish are popular pets for beginners, requiring relatively simple care. [{'source': 'fish-pets-doc'}]


In [35]:
results = vector_store.similarity_search_with_score(
    "Cats are independent pets that often enjoy their own space.",
    k=2,
    filter={"source": "mammal-pets-doc"}
)

for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")


* [SIM=0.827372] Cats are independent pets that often enjoy their own space. [{'source': 'mammal-pets-doc'}]
* [SIM=0.626672] Rabbits are social animals that need plenty of space to hop around. [{'source': 'mammal-pets-doc'}]


In [19]:
#setup LLM
import langchain.chat_models
import langchain_google_genai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
import os
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [23]:
dir(genai)

['AqaInput',
 'AqaOutput',
 'ChatGoogleGenerativeAI',
 'DoesNotExistsException',
 'GenAIAqa',
 'GoogleGenerativeAI',
 'GoogleGenerativeAIEmbeddings',
 'GoogleVectorStore',
 'HarmBlockThreshold',
 'HarmCategory',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_common',
 '_enums',
 '_function_utils',
 '_genai_extension',
 '_image_utils',
 'chat_models',
 'embeddings',
 'genai_aqa',
 'google_vector_store',
 'llms']

In [24]:
#combine retriver and LLM
!pip install -qU langchain-community
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from langchain.chains import RetrievalQA
from uuid import uuid4

In [42]:

llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=os.environ['GEMINI_API_KEY'],
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [44]:
# setup for a function for llm to answer based on the rag database we set earlier
def answer_to_user_by_model(query: str):
  result = vector_store.similarity_search_with_score(query, k=2)
  print(len(result))
  final_answer = llm.invoke(f"ANSWER THIS USER QUERY:{query},HERE ARE SOME REFERENCE ANSWER{result}")
  return final_answer


In [45]:
# Calling the function with a query that is in the database if a query outside the database is asked it won't be able to answer
response = answer_to_user_by_model("tell me about cats")
print(response.content)


2
Based on the provided text, cats are independent pets that often enjoy their own space.  The second document is about dogs and is irrelevant to the query about cats.

